###НАПИШЕМ РЕКОМЕНДАТЕЛЬНУЮ СИСТЕМУ, КОТОРАЯ НА МУЗЫКАЛЬНУЮ КОМПОЗИЦИЮ БУДЕТ  ДАВАТЬ ПОХОЖИЕ ПО ОЦЕНКАМ ПОЛЬЗОВАТЕЛЕЙ

данные будут предрассчитаны и записаны на Amazon S3, откуда впоследствии будут вызываться через эндпоинт на Amazon API Gateway + Lambda

Данильченко Вадим

In [1]:
import numpy as np
from multiprocessing import Pool
import pickle
import numpy as np
from tqdm import tqdm_notebook, tqdm
from scipy.spatial.distance import chebyshev, minkowski , cosine


import sqlalchemy
#import psycopg2
from pandas import DataFrame

import pandas as pd
import numpy as np
import matplotlib as plt
import pyarrow as pa
import ast, json
import re
from tqdm import tqdm, trange

скачаем датасет

In [2]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Alternative_Rock.json.gz

--2021-10-20 17:19:08--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Alternative_Rock.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2170570 (2.1M) [application/x-gzip]
Saving to: ‘reviews_Alternative_Rock.json.gz’

reviews_Alternative 100%[===================>]   2.07M  2.95MB/s    in 0.7s    

2021-10-20 17:19:09 (2.95 MB/s) - ‘reviews_Alternative_Rock.json.gz’ saved [2170570/2170570]



In [3]:
!ls

reviews_Alternative_Rock.json  reviews_Alternative_Rock.json.gz  sample_data


In [4]:
!gzip -d reviews_Alternative_Rock.json.gz

gzip: reviews_Alternative_Rock.json already exists; do you wish to overwrite (y or n)? y


In [5]:
df = pd.read_json('/content/reviews_Alternative_Rock.json',lines=True)#.sample(1000)
print('dataframe shape:', df.shape)
df.head(5)

dataframe shape: (8511, 9)


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A5K6D3S4RPEJD,B000QQPH06,Imagi Nation,"[1, 1]","WOW, WOW, WOW. WOW, WOW. Endlessly fascinating...",5,WOW.,1350518400,"10 18, 2012"
1,A3VKOIIOZWSUS6,B000QQPNV4,darren griffiths,"[0, 0]",I love this disc! It reminds me of Pele and P...,5,If not for Propagahndi...,1392249600,"02 13, 2014"
2,A3EDT3R9U9DYCS,B000QQRQ8C,artstar73,"[0, 0]",Just stumbled across this band while checking ...,5,Refreshing...,1404259200,"07 2, 2014"
3,AVK8OZGAE6YP5,B000QQTBFS,"J. Pate ""frankenhooker""","[2, 2]",This album is a healthy serving of sleaze. Th...,5,The sound of Satan's fists beating Santa Claus,1224547200,"10 21, 2008"
4,A3CEMZCPQSMRRQ,B000QQTJ1Y,Curtis,"[1, 1]",I have this CD and I find myself playing it al...,5,A Very Cool CD,1258156800,"11 14, 2009"


выделим необходимые фичи и преобразуем в матричный вид, где по строкам будет id оцениваемого произведения, по столбцам id оценивающего, а на пересечении оценка

In [6]:
df_matrix = df[['asin','reviewerID','overall']].drop_duplicates()
df_matrix = df_matrix.pivot(index='asin',columns='reviewerID',values='overall').fillna(0)
print('matrix shape:', df_matrix.shape)
df_matrix.head()

matrix shape: (3842, 7777)


reviewerID,A00532872P585BKPLQWE1,A01429103VG589HFBE2CC,A03091402ZJS17VCR8H4S,A066501224MIVO2Z9ACKL,A06662542P9IP59TYGWO6,A103H2JKAP9UGK,A10426MG5YEQEK,A104CBA4A4HYOF,A104XP70U87YYJ,A106CE5448O1SX,A106V556GJZIMV,A108AE1VHGMZZQ,A108AIBHE5OQJE,A108Q1IW9HDUH2,A108S1URV9XFHH,A109KSUWBE5TXE,A109O1883P9RUL,A10B71NYJBS6PE,A10BPI2J9WBM3T,A10CHHWKDGZ9YK,A10D0I6EVTURIJ,A10D0ZHRIT44CX,A10D2KGBCFR8VN,A10DV8R7H1K93V,A10DWRCYPP5TO3,A10FDA1OV63X2X,A10FL7ST0C41W3,A10G758EDK9TTE,A10H6S10QP7WV3,A10HIUT4KSJEHJ,A10IAABI6MXLFA,A10IADXIZBY2B2,A10IEEHNYQQH36,A10IW7X6DSEMXW,A10JIPUYPPP8PS,A10JZDGVEBSNMP,A10KCGSDJJAVRO,A10KIIMLNQKIB,A10N2RHM1N1Q14,A10NCINJJ2F8NH,...,AZ9B3MP8H63PM,AZA2SCH7U4JPZ,AZAE9U5GQFAM,AZAXZASQGK5W1,AZCPWGQY1KYL5,AZE78Q054OI0O,AZE7E0PVDXS8,AZEJGBO0B4N5P,AZF9NDW08PLPZ,AZFELU2ZTLRP5,AZFY6O5QL469D,AZG9JM60FHD2B,AZGGD8L1E2VY3,AZJSR345OX2C2,AZK1OWDQUXPTR,AZKFLKWBXQYNT,AZKXH0J59TK11,AZLDSNZZ4MD30,AZLL08XMPTGS3,AZLXQLP6NLJBP,AZLZ3004FZTRV,AZN22M2ZG0Z5Y,AZNCZNME0LHCE,AZNMMTGGGYIXW,AZNOS41BIVRTN,AZNUHQSHZHSUE,AZNYT1FQM24FB,AZPUC26MC7525,AZQY4AGVRDP2A,AZR1RBKOE6XDN,AZR40CEPXPL9,AZSMRGY9RSACI,AZTQHOXVFE8FL,AZUA2OFAQCTB8,AZVE8F4CZUMNE,AZVP7V02ZVWQB,AZWHBFU7UJF41,AZZ5AADBV2YG5,AZZH379R4HG52,AZZWPNME0GQZ2
asin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
B000QQPH06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000QQPNV4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000QQRQ8C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000QQTBFS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000QQTJ1Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


посчитаем также среднюю оценку по композиции и последнюю дату отзыва 

In [7]:
df_features = df.groupby('asin').agg({'overall':'mean','reviewTime':'max'})
df_features.head()

,overall,reviewTime
asin,,
B000QQPH06,5.0,"10 18, 2012"
B000QQPNV4,5.0,"02 13, 2014"
B000QQRQ8C,5.0,"07 2, 2014"
B000QQTBFS,5.0,"10 21, 2008"
B000QQTJ1Y,5.0,"11 14, 2009"


для каждого произведения веткором является вектор оценок пользователей, но вектор большой и не разреженный, нельзя его напрямую использовать -> уменьшим размерность до 15, используя алгоритм Non-Negative Matrix Factorization, после чего посчитаем евклидово расстояние между векторами 

In [8]:
from sklearn.decomposition import NMF
model = NMF(n_components=15, init='random', random_state=777)
W = model.fit_transform(df_matrix)

In [9]:
from sklearn.metrics import euclidean_distances
dist_m = euclidean_distances(W)

In [10]:
# посмотрим 20 наиболее близких вектора для первой композиции
products = df_matrix.iloc[dist_m[0].argsort()[:20]].index.values
df_features.loc[products]

,overall,reviewTime
asin,,
B000QQPH06,5.000000,"10 18, 2012"
B004YX1152,4.500000,"11 10, 2012"
B003NN9NRW,4.000000,"07 20, 2013"
B00CWQAZK0,3.666667,"08 20, 2013"
B009CO6MU2,4.000000,"06 7, 2013"
B00FE8T1TE,4.500000,"11 2, 2013"
B001PN2VV2,4.500000,"09 12, 2009"
B00DJRLDHU,4.500000,"07 5, 2014"
B000QZZIDI,4.500000,"09 7, 2012"


напишем функцию, которая будет давать рекомендации по нашему id (номер строки матрицы) в формате, близкому к ответу будущего Сервиса

In [11]:
def get_recommendations(_id, cnt):

  elements = df_matrix.iloc[dist_m[_id].argsort()[:cnt]].index.values
  df_output = df_features.loc[elements]


  similar_elements = [str(x) for x in df_output.index.astype('str').values.tolist()]
  dates = [str(x) for x in df_output.reviewTime.astype('str').values.tolist()]
  scores = [float(x) for x in df_output['overall'].astype('float32').values.tolist()]
  output_list = [similar_elements,dates,scores]


  return  {
      "data": [
          {
              "similar_elements": similar_elements,
              "sku_rating": scores,
              "last_reviewed": dates
          }
      ],
  "statusCode": 200
  }

get_recommendations(20, 5)

{'data': [{'last_reviewed': ['06 22, 2009',
    '11 1, 2012',
    '08 17, 2013',
    '11 17, 2012',
    '11 27, 2007'],
   'similar_elements': ['B000QQWIUI',
    'B009SZAZC6',
    'B004F7DLR8',
    'B003IU1FLW',
    'B000VL7LVY'],
   'sku_rating': [5.0, 5.0, 5.0, 5.0, 5.0]}],
 'statusCode': 200}

предрасчитаем рекомендации для каждой композиции и загрузим на AWS S3

In [12]:
!pip install boto3

In [13]:
AWS_SERVER_PUBLIC_KEY = 'AKIAS...'
AWS_SERVER_SECRET_KEY = 'v3b7yn...'

In [14]:
 import boto3
 s3_client = boto3.client('s3', 
                      aws_access_key_id=AWS_SERVER_PUBLIC_KEY, 
                      aws_secret_access_key=AWS_SERVER_SECRET_KEY
                      )

In [15]:
# проверим возможность добавлять элементы на AWS s3 на одном примере
s3_client.put_object(Bucket='lambda-recommendations-test',Key=str(0)+'_v1',Body=pickle.dumps(get_recommendations(0, 50)))

{'ETag': '"f80182319c6ca04c6ac1ce1fbd43a574"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Wed, 20 Oct 2021 17:19:50 GMT',
   'etag': '"f80182319c6ca04c6ac1ce1fbd43a574"',
   'server': 'AmazonS3',
   'x-amz-id-2': '3K+QF1rZcukdSNQ1RZZS9iHoYqCNx9VTdEmu4u0dJ28z2ntBIcZKysPwhfsXhcQ3dLjVVSYYZ8E=',
   'x-amz-request-id': 'QYP0NWJ68YT19JRG'},
  'HTTPStatusCode': 200,
  'HostId': '3K+QF1rZcukdSNQ1RZZS9iHoYqCNx9VTdEmu4u0dJ28z2ntBIcZKysPwhfsXhcQ3dLjVVSYYZ8E=',
  'RequestId': 'QYP0NWJ68YT19JRG',
  'RetryAttempts': 1}}

In [16]:
str(0)+'_v1'

'0_v1'

In [17]:
# прочитаем записанный нами файл
pickle.loads(s3_client.get_object(Bucket='lambda-recommendations-test',Key="0"+'_v1')['Body'].read())

{'data': [{'last_reviewed': ['10 18, 2012',
    '11 10, 2012',
    '07 20, 2013',
    '08 20, 2013',
    '06 7, 2013',
    '11 2, 2013',
    '09 12, 2009',
    '07 5, 2014',
    '09 7, 2012',
    '07 19, 2013',
    '08 3, 2009',
    '12 8, 2010',
    '08 24, 2013',
    '07 28, 2011',
    '10 11, 2011',
    '06 22, 2013',
    '10 16, 2009',
    '11 12, 2013',
    '08 22, 2010',
    '12 7, 2008',
    '01 22, 2009',
    '10 10, 2011',
    '10 10, 2011',
    '08 24, 2013',
    '05 22, 2012',
    '11 28, 2012',
    '06 26, 2013',
    '06 1, 2014',
    '05 25, 2012',
    '10 1, 2012',
    '12 21, 2012',
    '01 12, 2013',
    '10 21, 2012',
    '11 19, 2012',
    '05 18, 2012',
    '05 8, 2012',
    '01 18, 2013',
    '03 18, 2013',
    '11 8, 2013',
    '07 8, 2012',
    '06 25, 2013',
    '08 2, 2012',
    '09 2, 2013',
    '02 1, 2014',
    '10 26, 2012',
    '05 13, 2012',
    '01 27, 2013',
    '10 8, 2013',
    '05 28, 2012',
    '04 20, 2014'],
   'similar_elements': ['B000QQPH06',
  

In [18]:
# проверим соответствует ли то, что мы записали на S3 тому же, что нам выдает локально
get_recommendations(0, 50)==pickle.loads(s3_client.get_object(Bucket='lambda-recommendations-test',Key="0"+'_v1')['Body'].read())

True

In [20]:
#  запишем все рекомендации по порядку

for idx in trange(dist_m.shape[0]):
  try:
      s3_client.put_object(Bucket='lambda-recommendations-test',Key=str(idx)+'_v1',Body=pickle.dumps(get_recommendations(idx, 50)))
  except:
      print('не записался: ', idx)

100%|██████████| 3842/3842 [04:07<00:00, 15.55it/s]
